<a href="https://colab.research.google.com/github/SYEON9/natural_language_3th/blob/main/NLP/02_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Word2Vcec

1. 주어진 단어들을 word2vec 모델에 들어갈 수 있는 형태로 만든다.
2. CBOW, Skip-gram 모델을 각각 구현한다.
3. 모델을 실제로 학습하고 결과를 확인한다. 
4. 산점도를 그려 단어들의 대략적인 위치를 확인한다. 



먼저 Word2Vec에 대해 간단하게 알아보자.

One-hot vector는 단어 벡터 간 유의미한 유사도를 계산할 수 없다. 그래서 단어 벡터 간 유의미한 유사도를 반영할 수 있도록 단어의 의미를 수치화 할 수 있는 방법이 필요하다. 이를 위해 사용되는 대표적인 방법이 Word2Vec이다. 

word2vec을 실행하기 위해서는 우선 수치화된 단어가 필요하다. 그럼 단어를 수치화 하는 것을 뭐라고 할까?

다차원 공간에 단어의 의미를 벡터화하는 방법을 분산 표현이라고 하고 분산 표현을 이용하여 단어 간 의미적 유사성을 벡터화 하는 작업 == word embedding. 이것으로 표현된 벡터== embedding vector.

분산 표현은 저차원에 단어의 의미를 여러 차원에 분산하여 표현하므로 단어 벡터 간 유의미한 유사도를 계샇날 수 있다. 

word2vec에서는 비슷한 문맥에서 등장하는 단어들은 비슷한 의미를 가진다. 즉, 각 단어 벡터가 유사한 벡터값을 가진다는 의미이다.  
___

### CBOW vs Skip-gram

Word2Vec의 학습 방식에는 CBOW, Skip-gram 두가지가 있다. 두 방법의 메커니즘 자체는 비슷하다. 

- CBOW: 주변 단어를 입력으로 중간 단어를 예측.
- Skip-Gram: 중간 단어를 입력으로 주변 단어를 예측.



### 필요패키지 import

In [ ]:
#colab에서 nanum font 사용하기.
#폰트 설치
!sudo apt-get install -y fonts-nanum    #nanum font install
!sudo fc-cache -fv                      #ubuntu font install
!rm ~/. cache/matplotlib -rf            #matplotlib이 사용할 font 정보를 삭제..?

In [ ]:
#한국어 처리 패키지(konlpy) 설치
!pip install konlpy

In [ ]:
from tqdm import tqdm
from konlpy.tag import Mecab, Twitter, Okt, Kkma
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

import torch
import copy
import numpy as np

### 데이터 전처리

데이터를 확인하고 Word2Vec 형식에 맞게 전처리한다. 

In [ ]:
train_data = [
  "정말 맛있습니다. 추천합니다.",
  "기대했던 것보단 별로였네요.",
  "다 좋은데 가격이 너무 비싸서 다시 가고 싶다는 생각이 안 드네요.",
  "완전 최고입니다! 재방문 의사 있습니다.",
  "음식도 서비스도 다 만족스러웠습니다.",
  "위생 상태가 좀 별로였습니다. 좀 더 개선되기를 바랍니다.",
  "맛도 좋았고 직원분들 서비스도 너무 친절했습니다.",
  "기념일에 방문했는데 음식도 분위기도 서비스도 다 좋았습니다.",
  "전반적으로 음식이 너무 짰습니다. 저는 별로였네요.",
  "위생에 조금 더 신경 썼으면 좋겠습니다. 조금 불쾌했습니다."
]

test_words = ['음식','맛','서비스','위생','가격']

Tokenization과 vocab을 만드는 과정은 이전 실습과 유사하다.

In [ ]:
tokenizer = Okt()

In [ ]:
# 토큰화하는 함수 make_tokenized를 만들자.
def make_tokenized(data):
    tokenized = []
    for sent in tqdm(data):
        #텍스트를 형태소 단위로 나눈다. stem을 사용해 어간을 추출한다.
        tokens = tokenizer.morphs(sent, stem = True)     
        tokenized.append(tokens)
    
    return tokenized

In [ ]:
#문장을 토큰화하자.
train_tokenized = make_tokenized(train_data)

In [ ]:
train_tokenized

In [ ]:
# 각 토큰들이 각각 몇개씩 존재하는지 세자.
word_count = defaultdict(int)    #int값이 default인 딕셔너리 생성.

for tokens in tqdm(train_tokenized):
    for token in tokens:
        word_count[token] += 1


In [ ]:
print(list(word_count))

In [ ]:
# 등장횟수가 많은 순서대로 정렬하여 확인하자. 
word_count = sorted(word_count.items(), key=lambda x:x[1], reverse = True)
print(list(word_count))

In [ ]:
#만들어진 token을 이용하여 각 token에 index를 부여하자. 

w2i = {}
for pair in tqdm(word_count):
    if pair[0] not in w2i:
        w2i[pair[0]] = len(w2i)

i2w = {v:k for k,v in w2i.items()}

In [ ]:
print(train_tokenized)
print(w2i)

### CBOW

CBOW는 주변단어를 이용해 주어진 단어(=중심단어)를 예측하는 방법이다. 그러므로 몇개의 주변단어를 참고할지 정하는 window에 따라 input의 개수가 달라진다. 출력은 중심단어 하나이다. 


참고자료

* https://simonezz.tistory.com/35 
* https://towardsdatascience.com/nlp-101-word2vec-skip-gram-and-cbow-93512ee24314 

실제 모델에 들어가기위한 input을 만들기위해 Dataset 클래스를 정의한다. 

In [ ]:
class CBOWDataset(Dataset):
    def __init__(self, train_tokenized, window_size=2):
        self.x = []   #input word
        self.y = []   #target word

        
        for tokens in tqdm(train_tokenized):
            #token_ids: token의 id를 저장.
            token_ids = [w2i[token] for token in tokens]
            print(token_ids) 

            for i, id in enumerate(token_ids):
                if i-window_size >= 0 and i+window_size <len(token_ids):
                    #찾으려는 i를 제외하고 input에 필요한 데이터 범위 설정. 
                    self.x.append(token_ids[i-window_size:i] + token_ids[i+1:i+window_size+1])
                    self.y.append(id)

            self.x = torch.LongTensor(self.x)     #(전체 데이터 개수, 2*window_size)
            self.y = torch.LongTensor(self.y)     #(전체 데이터 개수)

    
    def __len__(self):
        return self.x.shape[1]

    def __gititem__(self, idx):
        return self.x[idx], self.y[idx]


In [ ]:
class CBOWDataset(Dataset):
  def __init__(self, train_tokenized, window_size=2):
    self.x = [] # input word
    self.y = [] # target word

    for tokens in tqdm(train_tokenized):
      #token_ids: token의 id를 저장
      token_ids = [w2i[token] for token in tokens]
      print(token_ids)

      for i, id in enumerate(token_ids):
        if i-window_size >= 0 and i+window_size < len(token_ids):
            #찾으려는 i를 제외하고 input에 필요한 데이터 범위 설정
          self.x.append(token_ids[i-window_size:i] + token_ids[i+1:i+window_size+1])
          self.y.append(id)

    self.x = torch.LongTensor(self.x)  # (전체 데이터 개수, 2 * window_size)
    self.y = torch.LongTensor(self.y)  # (전체 데이터 개수)

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [ ]:
# CBOW의 Dataset 객체 생성.
cbow_set = CBOWDataset(train_tokenized)
print(list(cbow_set))

####모델 Class 구현

CBOW Word2Vec 모델을 구현한다.
* `self.embedding`: `vocab_size` 크기의 one-hot vector를 트적 크기의 `dim` 차원으로 embedding 시키는 layer.
* `self.linear`: 변환된 embedding vector를 다시 원래 `vocab_size`로 바꾸는 layer.

In [ ]:
class CBOW(nn.Module):
  def __init__(self, vocab_size, dim):
    super(CBOW, self).__init__()
    self.embedding = nn.Embedding(vocab_size, dim, sparse=True)
    self.linear = nn.Linear(dim, vocab_size)

  # B: batch size, W: window size, d_w: word embedding size, V: vocab size
  def forward(self, x):  # x: (B, 2W)
    embeddings = self.embedding(x)  # (B, 2W, d_w)
    embeddings = torch.sum(embeddings, dim=1)  # (B, d_w)
    output = self.linear(embeddings)  # (B, V)
    return output

CBOW 모델을 생성한다. 

In [ ]:
cbow = CBOW(vocab_size = len(w2i), dim = 256)

#### 모델 학습

다음과 같이 hyperparameter를 세팅하고 `DataLoader` 객체를 만든다. 

In [ ]:
batch_size=4
learning_rate = 5e-4
num_epochs = 5
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

cbow_loader = DataLoader(cbow_set, batch_size=batch_size)

CBOW 모델 학습

In [ ]:
cbow.train()
cbow = cbow.to(device)
optim = torch.optim.SGD(cbow.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

for e in range(1, num_epochs+1):
  print("#" * 50)
  print(f"Epoch: {e}")
  
  for batch in tqdm(cbow_loader):
    x, y = batch
    x, y = x.to(device), y.to(device) # (B, W), (B)
    output = cbow(x)  # (B, V)
 
    optim.zero_grad()
    loss = loss_function(output, y)
    loss.backward()
    optim.step()

    print(f"Train loss: {loss.item()}")

print("Finished.")

### Skip-gram

중심 단어를 이용하여 주변 단어를 예측하는 방법이다. 데이터셋을 구성할 때, input x와 target y를 어떻게 설정하는지 CBOW와 비교하면서 살펴보자. 

input을 만들기 위해 Dataset 클래스를 정의한다. 

In [ ]:
class SkipGramDataset(Dataset):
  def __init__(self, train_tokenized, window_size=2):
    self.x = []
    self.y = []

    for tokens in tqdm(train_tokenized):
      token_ids = [w2i[token] for token in tokens]
      for i, id in enumerate(token_ids):
        if i-window_size >= 0 and i+window_size < len(token_ids):
          self.y += (token_ids[i-window_size:i] + token_ids[i+1:i+window_size+1])
          self.x += [id] * 2 * window_size

    self.x = torch.LongTensor(self.x)  # (전체 데이터 개수)
    self.y = torch.LongTensor(self.y)  # (전체 데이터 개수)

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

데이터 생성

In [ ]:
skipgram_set = SkipGramDataset(train_tokenized)
print(list(skipgram_set))

#### class 모델 구현하기

`self.embedding`: `vocab_size` 크기의 one-hot vector를 특정 크기의 `dim` 차원으로 embedding 시키는 layer.
*   `self.linear`: 변환된 embedding vector를 다시 원래 `vocab_size`로 바꾸는 layer.


In [ ]:
class SkipGram(nn.Module):
    
  def __init__(self, vocab_size, dim):
    super(SkipGram, self).__init__()
    self.embedding = nn.Embedding(vocab_size, dim, sparse=True)
    self.linear = nn.Linear(dim, vocab_size)

  # B: batch size, W: window size, d_w: word embedding size, V: vocab size
  def forward(self, x): # x: (B)
    embeddings = self.embedding(x)  # (B, d_w)
    output = self.linear(embeddings)  # (B, V)
    return output

모델 생성

In [ ]:
skipgram = SkipGram(vocab_size = len(w2i), dim = 256)

#### 모델 학습

hyperparameter를 세팅하고 DataLoader 객체를 만든다. 

In [ ]:
batch_size=4
learning_rate = 5e-4
num_epochs = 5
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

skipgram_loader = DataLoader(skipgram_set, batch_size = batch_size)

다음은 SkipGram 모델 학습이다.

In [ ]:
skipgram.train()
skipgram = skipgram.to(device)
optim = torch.optim.SGD(skipgram.parameters(), lr = learning_rate)
loss_function = nn.CrossEntropyLoss()

for e in range(1, num_epochs+1):
    print("#"*50)
    print(f"Epoch:{e}")

    for batch in tqdm(skipgram_loader):
        x, y = batch    #각각 정보를 넣음.
        x, y = x.to(device), y.to(device)
        output = skipgram(x)

        # train
        optim.zero_grad()
        loss = loss_function(output, y)
        loss.backward()
        optim.step()

    print(f"Train loss:{loss.item()}")
print("Finished.")


### 테스트

학습된 각 모델을 이용하여 test 단어들의 word embedding을 확인하자.

In [ ]:
#CBOW

for word in test_words:
    input_id = torch.LongTensor([w2i[word]]).to(device)
    emb = cbow.embedding(input_id)

    print(f"Word: {word}")
    print(emb.squeeze(0))

In [ ]:
# SkipGram

for word in test_words:
    input_id = torch.LongTensor([w2i[word]]).to(device)
    emb = skipgram.embedding(input_id)

    print(f"Word: {word}")
    print(max(emb.squeeze(0)))

In [ ]:
test_words

In [ ]:
i2w[35]

In [ ]:
# similarity

def most_similar(word, top_k = 5):
    input_id = torch.LongTensor([w2i[word]]).to(device)
    input_emb = skipgram.embedding(input_id)
    score = torch.matmul(input_emb, skipgram.embedding.weight.transpose(1,0)).view(-1)

    _, top_k_ids = torch.topk(score, top_k)    #주어진 결과 중에서 가장 확률이 높은 값을 top_k개 반환

    return [i2w[word_id.item()] for word_id in top_k_ids][1:]

In [ ]:
most_similar('가격')

### Word2Vec 시각화

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

#matplotlib의 한글 깨짐 처리 시작
plt.rc('font', family='NanumBarunGothic')


In [ ]:
pca = PCA(n_components=2)

In [ ]:
pc_weight = pca.fit_transform(skipgram.embedding.weight.data.cpu().numpy())

In [ ]:
plt.figure(figsize = (15,15))

for word_id,(x_coordinate,y_coordinate) in enumerate(pc_weight):
  plt.scatter(x_coordinate,y_coordinate,color="blue")
  plt.annotate(i2w[word_id], (x_coordinate, y_coordinate))